In [2]:
import pandas as pd
import networkx as nx
from sklearn.cluster import KMeans
import numpy as np
from sklearn.impute import SimpleImputer
import plotly.express as px


file_path = '/Users/gabesmithline/Desktop/PettingZoo-MPE-Pursuit/AFRL/simulation_data_with_features.csv'
simulation_data = pd.read_csv(file_path)

# Function to parse feature arrays from strings to lists of floats
def parse_feature_arrays(feature):
    if isinstance(feature, str):
        return [float(x) for x in feature.strip('[]').split(',')]
    return feature

# Apply the parsing function to the relevant columns
simulation_data['distances_to_agents'] = simulation_data['distances_to_agents'].apply(parse_feature_arrays)
simulation_data['angles_to_agents'] = simulation_data['angles_to_agents'].apply(parse_feature_arrays)
simulation_data['self_pos'] = simulation_data['self_pos'].apply(parse_feature_arrays)
simulation_data['action'] = simulation_data['action'].apply(parse_feature_arrays)

# Filter out the evader
pursuers_data = simulation_data[simulation_data['agent'] != 'agent_0']
evader_data = simulation_data[simulation_data['agent'] == 'agent_0']



In [3]:
# Function to compute distance to evader
def compute_distance_to_evader(pursuer_pos, evader_pos):
    return np.linalg.norm(np.array(pursuer_pos) - np.array(evader_pos))

# Function to compute angle to evader
def compute_angle_to_evader(pursuer_pos, evader_pos):
    delta = np.array(evader_pos) - np.array(pursuer_pos)
    return np.arctan2(delta[1], delta[0])

# Compute distances and angles to evader for each pursuer
evader_pos = evader_data.iloc[0]['self_pos']  # Assuming there's only one evader and it's the first row
pursuers_data['distance_to_evader'] = pursuers_data['self_pos'].apply(lambda x: compute_distance_to_evader(x, evader_pos))
pursuers_data['angle_to_evader'] = pursuers_data['self_pos'].apply(lambda x: compute_angle_to_evader(x, evader_pos))

# Function to compute mean distance to other pursuers
def compute_mean_distance_to_others(distances):
    return np.mean(distances)

# Compute mean distances to other pursuers for each pursuer
pursuers_data['mean_distance_to_pursuers'] = pursuers_data['distances_to_agents'].apply(compute_mean_distance_to_others)

# Combine relevant features into a single feature matrix
feature_matrix = np.hstack((
    np.vstack(pursuers_data['mean_distance_to_pursuers']),
    np.vstack(pursuers_data['angles_to_agents']),
    np.vstack(pursuers_data['distance_to_evader']),
    np.vstack(pursuers_data['angle_to_evader'])
))

print("Feature matrix shape:", feature_matrix.shape)

Feature matrix shape: (8750, 9)


/var/folders/fh/fwc37qhn04d8sxp65hwv1kxm0000gn/T/ipykernel_77599/2047733710.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  pursuers_data['distance_to_evader'] = pursuers_data['self_pos'].apply(lambda x: compute_distance_to_evader(x, evader_pos))
/var/folders/fh/fwc37qhn04d8sxp65hwv1kxm0000gn/T/ipykernel_77599/2047733710.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  pursuers_data['angle_to_evader'] = pursuers_data['self_pos'].apply(lambda x: compute_angle_to_evader(x, evader_pos))
/var/folders/f

In [4]:
from sklearn.cluster import KMeans
kmeans = KMeans(n_clusters=2, random_state=0).fit(feature_matrix)
cluster_labels = kmeans.labels_

# Add the cluster labels to the dataframe
pursuers_data['cluster'] = cluster_labels

# Create a DataFrame for plotting
plot_data = pd.DataFrame({
    'mean_distance_to_pursuers': pursuers_data['mean_distance_to_pursuers'],
    'angles': pursuers_data['angles_to_agents'].apply(lambda x: np.mean(x)),  # Use mean angle to simplify
    'distance_to_evader': pursuers_data['distance_to_evader'],
    'cluster': pursuers_data['cluster'],
    'agent': pursuers_data['agent'],
    'active_status': pursuers_data['agent_type'],
    'angle_to_evader': pursuers_data['angle_to_evader']


})

# Create the 4D scatter plot
fig = px.scatter_3d(plot_data, x='angle_to_evader', y='angles', z='distance_to_evader', color='cluster', 
                    hover_data=['agent', 'active_status'], size_max=18, opacity=0.7,
                    title='4D Clustering Visualization',
                    labels={'angle_to_evader': 'Angle to Evader', 'angles': 'Mean Angles to Pursuers', 'distance_to_evader': 'Distance to Evader'})

fig.update_layout(scene=dict(
                    xaxis_title='Mean Angle to Other Pursuers',
                    yaxis_title='Mean Distance to Pursuers',
                    zaxis_title='Angle to Evader'))
fig.show()


/Users/gabesmithline/miniconda3/lib/python3.10/site-packages/sklearn/cluster/_kmeans.py:1416: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  super()._check_params_vs_input(X, default_n_init=10)
/var/folders/fh/fwc37qhn04d8sxp65hwv1kxm0000gn/T/ipykernel_77599/2505754325.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  pursuers_data['cluster'] = cluster_labels


## Notes
- Broadly it seems as the angle to the evader increase the distance to the other pursuers decreases. This might indicate that as the angle to the evader increases, the pursuers tend to converge towards each other. It could suggest that the pursuers coordinate their movements to form tighter formations when the evader is at certain angles.  The pursuers might be using specific strategies to minimize the evader's escape routes or to maximize their chances of containment. Tighter formations can make it more difficult for the evader to slip through gaps between pursuers. 
- 

In [5]:
# Define weights for the features
w1, w2, w3, w4 = 1.0, 1.0, 1.0, 1.0  # You can adjust these weights

# Function to calculate combined feature distance
def combined_feature_distance_matrix(matrix):
    d1 = np.abs(matrix[:, None, 0] - matrix[None, :, 0])
    d2 = np.abs(matrix[:, None, 1] - matrix[None, :, 1])
    d3 = np.abs(matrix[:, None, 2] - matrix[None, :, 2])
    d4 = np.abs(matrix[:, None, 3] - matrix[None, :, 3])
    return w1 * d1 + w2 * d2 + w3 * d3 + w4 * d4

# Compute pairwise combined feature distances
distances = combined_feature_distance_matrix(feature_matrix)
pursuers_data.reset_index(drop=True, inplace=True)



In [6]:
# Check for missing values in the 'cluster' column
if pursuers_data['cluster'].isnull().any():
    print("Warning: Missing values in 'cluster' column.")
    pursuers_data['cluster'] = pursuers_data['cluster'].fillna(-1)  # Fill missing values with a default value


In [7]:
import networkx as nx

# Initialize an empty graph
G = nx.Graph()

# Add nodes with cluster labels and print node information
for idx, row in pursuers_data.iterrows():
    G.add_node(idx, cluster=row['cluster'], agent=row['agent'])
    print(f"Added node {idx} with cluster {row['cluster']}")

# Ensure all nodes have the 'cluster' attribute
for idx, row in pursuers_data.iterrows():
    if 'cluster' not in G.nodes[idx]:
        G.nodes[idx]['cluster'] = row['cluster']
        print(f"Setting missing cluster for node {idx} to {row['cluster']}")

# Print node information before adding edges
for node in G.nodes(data=True):
    print(f"Node {node[0]} has attributes {node[1]}")

# Define a threshold for creating edges
threshold_distance = np.mean(distances)  # Adjust this threshold if necessary

# Add edges based on combined feature distances
for i in range(len(pursuers_data)):
    for j in range(i + 1, len(pursuers_data)):
        if distances[i, j] < threshold_distance:
            G.add_edge(i, j, weight=distances[i, j])

# Verify that all nodes have the 'cluster' attribute after adding edges
for node in G.nodes(data=True):
    if 'cluster' not in node[1]:
        print(f"Node {node[0]} is missing 'cluster' attribute")


Added node 0 with cluster 1
Added node 1 with cluster 0
Added node 2 with cluster 1
Added node 3 with cluster 0
Added node 4 with cluster 1
Added node 5 with cluster 1
Added node 6 with cluster 0
Added node 7 with cluster 1
Added node 8 with cluster 0
Added node 9 with cluster 1
Added node 10 with cluster 1
Added node 11 with cluster 0
Added node 12 with cluster 1
Added node 13 with cluster 0
Added node 14 with cluster 1
Added node 15 with cluster 1
Added node 16 with cluster 0
Added node 17 with cluster 1
Added node 18 with cluster 0
Added node 19 with cluster 1
Added node 20 with cluster 1
Added node 21 with cluster 0
Added node 22 with cluster 1
Added node 23 with cluster 0
Added node 24 with cluster 1
Added node 25 with cluster 1
Added node 26 with cluster 0
Added node 27 with cluster 1
Added node 28 with cluster 0
Added node 29 with cluster 1
Added node 30 with cluster 1
Added node 31 with cluster 0
Added node 32 with cluster 1
Added node 33 with cluster 0
Added node 34 with clust

In [8]:
import matplotlib.pyplot as plt

# Draw the graph
pos = nx.spring_layout(G)
edge_weights = nx.get_edge_attributes(G, 'weight')
node_colors = [G.nodes[node]['cluster'] for node in G.nodes()]
nx.draw(G, pos, with_labels=True, node_color=node_colors, cmap=plt.cm.jet, node_size=500, font_size=10, font_color='white')
nx.draw_networkx_edge_labels(G, pos, edge_labels=edge_weights)
plt.title("Coordination Graph of Pursuers")
plt.show()


Matplotlib created a temporary cache directory at /var/folders/fh/fwc37qhn04d8sxp65hwv1kxm0000gn/T/matplotlib-dnc7gn2h because the default path (/Users/gabesmithline/.matplotlib) is not a writable directory; it is highly recommended to set the MPLCONFIGDIR environment variable to a writable directory, in particular to speed up the import of Matplotlib and to better support multiprocessing.
Exception ignored in: <bound method IPythonKernel._clean_thread_parent_frames of <ipykernel.ipkernel.IPythonKernel object at 0x107d3d690>>
Traceback (most recent call last):
  File "/Users/gabesmithline/miniconda3/lib/python3.10/site-packages/ipykernel/ipkernel.py", line 770, in _clean_thread_parent_frames
    def _clean_thread_parent_frames(
KeyboardInterrupt: 
